# Laboratorio de Amazon Rekognition

En este laboratorio exploraremos las siguientes capacidades de Rekognition.

- Detección de Objetos y Escenas en imágenes.
- Análisis Facial
- Comparación de rostros
- Detección de equipo protector

*Tip: Durante los laboratorios es posible cambiar las URLs de las imagenes para experimentar con otros archivos.*

## Detección de Objetos y Escenas

En esta sección tomaremos diferentes imágenes y las analizaremos utilizando el servicio de Rekognition.

Empezaremos por importar las librerías necesarias para la manipulación de imágenes y el cliente para utilizar Rekognition.

In [ ]:
import boto3
import requests
import io
from IPython.display import Image, display
from PIL import Image as PImage, ImageDraw, ImageFont

# Se obtiene el cliente de Rekognition.
rekognition = boto3.client('rekognition')

Esta función nos servirá para crear rectángulos al rededor de los objetos detectados.

In [ ]:
def box_labels(image_bytes, labels, confidence):
    font = ImageFont.load_default()
    image = PImage.open(io.BytesIO(image_bytes))
    width, height = image.size
    draw = ImageDraw.Draw(image)
    for label in labels:
        name = label['Name']
        if label['Confidence'] >= confidence:
            for instance in label['Instances']:
                if instance['Confidence'] >= confidence and 'BoundingBox' in instance:
                    start_x = instance['BoundingBox']['Left'] * width
                    start_y = instance['BoundingBox']['Top'] * height
                    end_x = start_x + instance['BoundingBox']['Width'] * width
                    end_y = start_y + instance['BoundingBox']['Height'] * height
                    draw.rectangle([(start_x, start_y), (end_x, end_y)], outline='black', width=2)
                    draw.text((start_x + 5, start_y + 5), name, fill='black', font=font)
    image_out = io.BytesIO()
    image.save(image_out, 'JPEG')
    return image_out.getvalue()

A continuación descargaremos las imágenes que serán utilizadas para detectar objetos y para detectar la escena.

In [ ]:
objects_url = 'https://dhei5unw3vrsx.cloudfront.net/images/skateboard_resized.jpg'
scene_url = 'https://dhei5unw3vrsx.cloudfront.net/images/landscape_resized.jpg'
objects_response = requests.get(objects_url, allow_redirects=True)
scene_response = requests.get(scene_url, allow_redirects=True)

Ahora que contamos con las imágenes a analizar comencemos con la detección de objetos.

In [ ]:
objects_image = objects_response.content
display(Image(data=objects_image))

Generamos la petición de análisis a Rekognition y utilizamos nuestra función para marcar las etiquetas detectadas.

In [ ]:
objects_a = rekognition.detect_labels(Image={
    'Bytes': objects_image
})
## Set a high confidence to reduce number of boxes.
new_objects_image = box_labels(objects_image, objects_a['Labels'], 90.0)
display(Image(data=new_objects_image))

Ahora analizaremos el contexto de una imagen.

In [ ]:
scene_image = scene_response.content
display(Image(data=scene_image))

In [ ]:
objects_a = rekognition.detect_labels(Image={
    'Bytes': scene_image
})
confidence = 90.0
detected_labels = list()
for label in objects_a['Labels']:
    if label['Confidence'] >= confidence:
        detected_labels.append(label['Name'])
print(detected_labels)

## Análisis facial

Ahora realizaremos un análisis facial.

In [ ]:
single_face_url = 'https://dhei5unw3vrsx.cloudfront.net/images/drive_resized.jpg'
multi_face_url = 'https://dhei5unw3vrsx.cloudfront.net/images/family_resized.jpg'
single_face_image = requests.get(single_face_url, allow_redirects=True).content
multi_face_image = requests.get(multi_face_url, allow_redirects=True).content

Definiremos las siguientes funciones para crear un cuadro donde Rekognition detecto el rostro e imprimir los detalles de cada rostro.

In [ ]:
box_colors = ['red', 'green', 'blue', 'orange', 'yellow', 'black', 'grey', 'purple']

def box_face(image_bytes, faces):
    image = PImage.open(io.BytesIO(image_bytes))
    width, height = image.size
    draw = ImageDraw.Draw(image)
    idx = 0
    for face in faces:
        if 'BoundingBox' in face:
            start_x = face['BoundingBox']['Left'] * width
            start_y = face['BoundingBox']['Top'] * height
            end_x = start_x + face['BoundingBox']['Width'] * width
            end_y = start_y + face['BoundingBox']['Height'] * height
            draw.rectangle([(start_x, start_y), (end_x, end_y)], outline=box_colors[idx], width=2)
            idx = idx + 1 if idx + 1 < len(box_colors) else 0
    image_out = io.BytesIO()
    image.save(image_out, 'JPEG')
    return image_out.getvalue()

def features(faces, confidence):
    idx = 0
    for face in faces:
        print('Box color: ' + box_colors[idx])
        print('Age range from {} to {}'.format(face['AgeRange']['Low'], face['AgeRange']['High']))
        print('Smiling {} with confidence {}'.format(face['Smile']['Value'], face['Smile']['Confidence']))
        print('Smiling {} with confidence {}'.format(face['Eyeglasses']['Value'], face['Eyeglasses']['Confidence']))
        print('Smiling {} with confidence {}'.format(face['Sunglasses']['Value'], face['Sunglasses']['Confidence']))
        print('Smiling {} with confidence {}'.format(face['Gender']['Value'], face['Gender']['Confidence']))
        print('Smiling {} with confidence {}'.format(face['Beard']['Value'], face['Beard']['Confidence']))
        print('Smiling {} with confidence {}'.format(face['Mustache']['Value'], face['Mustache']['Confidence']))
        print('Smiling {} with confidence {}'.format(face['EyesOpen']['Value'], face['EyesOpen']['Confidence']))
        print('Smiling {} with confidence {}'.format(face['MouthOpen']['Value'], face['MouthOpen']['Confidence']))
        for emotion in face['Emotions']:
            if emotion['Confidence'] >= confidence:
                print('The person is {}'.format(emotion['Type']))
        print('\n')
        idx = idx + 1 if idx + 1 < len(box_colors) else 0

En este primer ejercicio utilizaremos una imagen con un solo rostros para el análisis.

In [ ]:
display(Image(data=single_face_image))

In [ ]:
detect_face_r = rekognition.detect_faces(Image={'Bytes': single_face_image}, Attributes=['ALL'])
display(Image(data=box_face(single_face_image, detect_face_r['FaceDetails'])))
features(detect_face_r['FaceDetails'], 80.0)

Ahora realicemos el análisis sobre una imagen con diferentes rostros.

In [ ]:
display(Image(data=multi_face_image))

In [ ]:
detect_face_r = rekognition.detect_faces(Image={'Bytes': multi_face_image}, Attributes=['ALL'])
display(Image(data=box_face(multi_face_image, detect_face_r['FaceDetails'])))
features(detect_face_r['FaceDetails'], 80.0)

## Comparación de Rostros

En este laboratorio haremos una comparación de rostros entre dos imágenes con la finalidad de detectar si una persona se encuentra en ambas fotografías.

In [ ]:
source_img_url = 'https://dhei5unw3vrsx.cloudfront.net/images/source3_resized.jpg'
target_img_url = 'https://dhei5unw3vrsx.cloudfront.net/images/target3_resized.jpg'
source_image = requests.get(source_img_url, allow_redirects=True).content
target_image = requests.get(target_img_url, allow_redirects=True).content

display(Image(data=source_image))
display(Image(data=target_image))

Ahora realizamos el análisis con Rekognition.

In [ ]:
def crop_face(image_bytes, face):
    image = PImage.open(io.BytesIO(image_bytes))
    width, height = image.size
    cropped_bytes = image_bytes
    if 'BoundingBox' in face:
        start_x = face['BoundingBox']['Left'] * width
        start_y = face['BoundingBox']['Top'] * height
        end_x = start_x + face['BoundingBox']['Width'] * width
        end_y = start_y + face['BoundingBox']['Height'] * height
        cropped = image.crop((start_x, start_y, end_x, end_y))
        image_out = io.BytesIO()
        cropped.save(image_out, 'JPEG')
        cropped_bytes = image_out.getvalue()
    return cropped_bytes

def crop_faces(image_bytes, faces):
    cropped_faces = list()
    image = PImage.open(io.BytesIO(image_bytes))
    width, height = image.size
    for face in faces:
        if 'Face' in face and 'BoundingBox' in face['Face']:
            start_x = face['Face']['BoundingBox']['Left'] * width
            start_y = face['Face']['BoundingBox']['Top'] * height
            end_x = start_x + face['Face']['BoundingBox']['Width'] * width
            end_y = start_y + face['Face']['BoundingBox']['Height'] * height
            cropped = image.crop((start_x, start_y, end_x, end_y))
            image_out = io.BytesIO()
            cropped.save(image_out, 'JPEG')
            cropped_faces.append(image_out.getvalue())
        elif 'BoundingBox' in face:
            start_x = face['BoundingBox']['Left'] * width
            start_y = face['BoundingBox']['Top'] * height
            end_x = start_x + face['BoundingBox']['Width'] * width
            end_y = start_y + face['BoundingBox']['Height'] * height
            cropped = image.crop((start_x, start_y, end_x, end_y))
            image_out = io.BytesIO()
            cropped.save(image_out, 'JPEG')
            cropped_faces.append(image_out.getvalue())
    return cropped_faces

response = rekognition.compare_faces(SourceImage={'Bytes': source_image},
                                     TargetImage={'Bytes': target_image}) # Set to zero to recover al faces.
print('Face in the source image\n')
display(Image(data=crop_face(source_image, response['SourceImageFace'])))
print('Matched faces\n')
for face in crop_faces(target_image, response['FaceMatches']):
    display(Image(data=face))
print('Unmatched faces\n')
for face in crop_faces(target_image, response['UnmatchedFaces']):
    display(Image(data=face))

## Detección de equipo protector

En este laboratorio utilizaremos Rekognition para detectar equipo protector en imágenes.

In [ ]:
ppe_img_url = 'https://dhei5unw3vrsx.cloudfront.net/images/ppe_group_updated.jpg'
ppe_image = requests.get(ppe_img_url, allow_redirects=True).content
display(Image(data=ppe_image))

Ahora realizaremos el análisis de la imagen y marcaremos el equipo detectado.

In [ ]:
def ppe_box(image_bytes, persons):
    image = PImage.open(io.BytesIO(image_bytes))
    width, height = image.size
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    for person in persons:
        for body_part in person['BodyParts']:
            name = body_part['Name']
            for equipment in body_part['EquipmentDetections']:
                start_x = equipment['BoundingBox']['Left'] * width
                start_y = equipment['BoundingBox']['Top'] * height
                end_x = start_x + equipment['BoundingBox']['Width'] * width
                end_y = start_y + equipment['BoundingBox']['Height'] * height
                draw.rectangle([(start_x, start_y), (end_x, end_y)], outline='green', width=2)
                draw.text((start_x + 5, start_y + 5), name + ' ' + equipment['Type'], fill='black', font=font)
    image_out = io.BytesIO()
    image.save(image_out, 'JPEG')
    return image_out.getvalue()

ppe_response = rekognition.detect_protective_equipment(Image={'Bytes': ppe_image})
display(Image(data=ppe_box(ppe_image, ppe_response['Persons'])))